In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Referer': 'http://www.koreabaseball.com/Player/Search.aspx',
    'Origin': 'http://www.koreabaseball.com',
    'Content-Type': 'application/x-www-form-urlencoded',
}

In [3]:
rq = requests.Session()

In [4]:
src = BeautifulSoup(rq.get('http://www.koreabaseball.com/Player/Search.aspx', headers=headers).text,"lxml")
teams = [(team.text, team['value']) for team in src.find(class_='select02').find_all('option')][1:]

In [5]:
for x,y in teams:
    print (x, y)

두산 OB
삼성 SS
NC NC
넥센 WO
SK SK
한화 HH
KIA HT
롯데 LT
LG LG
kt KT


In [6]:
def make_post_next_pageset(code):
    return urllib.parse.urlencode({
            '__EVENTTARGET':'ctl00$ctl00$cphContainer$cphContents$ucPager$btnNext',
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': __VIEWSTATE,
            '__EVENTVALIDATION': __EVENTVALIDATION,
            '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
            'ctl00$ctl00$txtSearchWord': '',
            'ctl00$ctl00$cphContainer$cphContents$hfPage': __PREVPAGE,
            'ctl00$ctl00$cphContainer$cphContents$ddlTeam': str(code),
            'ctl00$ctl00$cphContainer$cphContents$ddlPosition': '',
            'ctl00$ctl00$cphContainer$cphContents$txtSearchPlayerName': ''
           })

In [7]:
def make_post_page(code, page):
    return urllib.parse.urlencode({
            '__EVENTTARGET':'ctl00$ctl00$cphContainer$cphContents$ucPager$btnNo' + str(page % 5 + 5*(page % 5 == 0)),
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': __VIEWSTATE,
            '__EVENTVALIDATION': __EVENTVALIDATION,
            '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
            'ctl00$ctl00$txtSearchWord': '',
            'ctl00$ctl00$cphContainer$cphContents$hfPage': __PREVPAGE,
            'ctl00$ctl00$cphContainer$cphContents$ddlTeam': str(code),
            'ctl00$ctl00$cphContainer$cphContents$ddlPosition': '',
            'ctl00$ctl00$cphContainer$cphContents$txtSearchPlayerName': ''
           })

In [8]:
def make_post_data(code, page=1):
    return urllib.parse.urlencode({
            '__EVENTTARGET':'ctl00$ctl00$cphContainer$cphContents$ddlTeam',
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': __VIEWSTATE,
            '__EVENTVALIDATION': __EVENTVALIDATION,
            '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
            'ctl00$ctl00$txtSearchWord': '',
            'ctl00$ctl00$cphContainer$cphContents$hfPage': '1',
            'ctl00$ctl00$cphContainer$cphContents$ddlTeam': str(code),
            'ctl00$ctl00$cphContainer$cphContents$ddlPosition': '',
            'ctl00$ctl00$cphContainer$cphContents$txtSearchPlayerName': ''
           })

In [9]:
def update_state(res, page=1):
    global __VIEWSTATE, __EVENTVALIDATION, __VIEWSTATEGENERATOR, __PREVPAGE
    src = BeautifulSoup(res,'lxml')
    if not src.find('input', {'name':'__VIEWSTATE'}):
        # Exception!!
        # print src
        pass
    __PREVPAGE = str(page)
    __VIEWSTATE = src.find('input', {'name':'__VIEWSTATE'})['value']
    __EVENTVALIDATION = src.find(id='__EVENTVALIDATION')['value']
    __VIEWSTATEGENERATOR = src.find(id='__VIEWSTATEGENERATOR')['value']

In [10]:
def get_player_list(code):
    rq = requests.Session()
    res = rq.get('http://www.koreabaseball.com/Player/Search.aspx', headers=headers)
    update_state(res.text)

    batters = []; pitchers = []

    for i in range(1,9):
        if i % 5 == 1:
            if i > 1:
                res = rq.post('http://www.koreabaseball.com/Player/Search.aspx', data=make_post_next_pageset(code), headers=headers)
            else:
                res = rq.post('http://www.koreabaseball.com/Player/Search.aspx', data=make_post_data(code, i), headers=headers)
            try:
                update_state(res.text)
            except:
                print ('Break at %d' % i)
                break
        res = rq.post('http://www.koreabaseball.com/Player/Search.aspx', data=make_post_page(code, i), headers=headers)
        try:
            update_state(res.text, i)
        except:
            print ('Break at %d' % i)
            break
        src = BeautifulSoup(res.text,'lxml')
        if src.find(class_='tEx') == None:
            print (code, i, make_post_page(code, i), res.text)
            break
        rows = src.find(class_='tEx').find('tbody').find_all('tr')
        print ('%s_%d: %d' % (code, i, len(rows)))
        if len(rows) == 0:
            break
        for j in range(len(rows)):
            link = rows[j].find('a')
            if link['href'].find('HitterDetail') != -1:
                batters.append(link['href'].split('playerId=')[-1])
            if link['href'].find('PitcherDetail') != -1:
                pitchers.append(link['href'].split('playerId=')[-1])
    return (batters, pitchers)

In [11]:
player_list = {
               'SS': None,
               'WO': None,
               'NC': None,
               'OB': None,
               'SK': None,
               'HH': None,
               'HT': None,
               'LT': None,
               'LG': None,
               'KT': None,
              }

In [12]:
for team in player_list.keys():
    player_list[team] = get_player_list(team)

OB_1: 20
OB_2: 20
OB_3: 20
OB_4: 20
OB_5: 16
Break at 6
WO_1: 20
WO_2: 20
WO_3: 20
WO_4: 20
WO_5: 13
Break at 6
NC_1: 20
NC_2: 20
NC_3: 20
NC_4: 20
NC_5: 5
Break at 6
HH_1: 20
HH_2: 20
HH_3: 20
HH_4: 20
HH_5: 20
HH_6: 7
Break at 7
HT_1: 20
HT_2: 20
HT_3: 20
HT_4: 20
HT_5: 20
HT_6: 9
Break at 7
LT_1: 20
LT_2: 20
LT_3: 20
LT_4: 20
LT_5: 17
Break at 6
LG_1: 20
LG_2: 20
LG_3: 20
LG_4: 20
LG_5: 13
Break at 6
KT_1: 20
KT_2: 20
KT_3: 20
KT_4: 20
KT_5: 3
Break at 6
SK_1: 20
SK_2: 20
SK_3: 20
SK_4: 20
SK_5: 6
Break at 6
SS_1: 20
SS_2: 20
SS_3: 20
SS_4: 20
SS_5: 19
Break at 6


In [13]:
player_list

{'HH': (['61700',
   '66710',
   '74745',
   '63744',
   '95158',
   '71610',
   '62768',
   '66715',
   '65709',
   '66749',
   '71752',
   '66704',
   '76746',
   '79705',
   '64704',
   '66740',
   '66714',
   '74756',
   '66707',
   '65705',
   '64724',
   '63703',
   '76753',
   '60404',
   '66719',
   '66708',
   '64086',
   '62797',
   '62748',
   '66713',
   '78756',
   '93732',
   '62770',
   '79869',
   '65703',
   '66702',
   '73136',
   '74148',
   '73725',
   '74163',
   '66718',
   '60667',
   '64110',
   '77654',
   '64706',
   '77464',
   '66720',
   '71347',
   '63700',
   '66706',
   '66716',
   '75808',
   '93802',
   '76720',
   '78745',
   '98144',
   '65707',
   '64717',
   '73602',
   '60805',
   '74729',
   '62700',
   '77243',
   '76404'],
  ['79748',
   '66705',
   '72447',
   '60461',
   '60757',
   '65764',
   '65769',
   '62710',
   '63900',
   '60764',
   '66741',
   '66701',
   '65742',
   '66748',
   '61767',
   '79760',
   '99737',
   '65724',
   '66709

In [14]:
def get_basic_info(src):
    items = src.find(class_='player_basic').find_all('li')
    dic = {}

    for item in items:
        dic[item.find('strong').text.replace(':', '')] = item.find('span').text

    return dic

In [15]:
def get_pitcher_info(id):
    url = 'http://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId=' + str(id)
    src = BeautifulSoup(rq.get(url, headers=headers).text,'lxml')

    tables = src.find_all(class_='tData01')[:2]
    ret = {}

    for table in tables:
        keys = [x.find('a')['title'].strip() if x and x.find('a') else x.text for x in table.find('thead').find_all('th')]
        values = [x.text for x in table.find('tbody').find_all('td')]

        if len(values) != len(keys) or values.count('-') > 0:
            return None

        fvalues = []

        def frac_to_float(val):
            return float(val.split('/')[0]) / float(val.split('/')[-1])

        for key, val in zip(keys, values):
            if key == u'팀명':
                fvalues.append(val)
                continue

            try:
                fvalues.append(float(val))
            except:
                if val.find(' ') != -1:
                    fvalues.append(sum([frac_to_float(x) if x.find('/') != -1 else float(x) for x in val.split()]))

        ret.update(dict(zip(keys, fvalues)))

    ret.update(get_basic_info(src))
    return ret

In [16]:
def get_batter_info(id):
    url = 'http://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=' + str(id)
    src = BeautifulSoup(rq.get(url, headers=headers).text)

    tables = src.find_all(class_='tData01')[:2]
    ret = {}

    for table in tables:
        keys = [x.find('a')['title'].strip() if x and x.find('a') else x.text for x in table.find('thead').find_all('th')]
        values = [x.text for x in table.find('tbody').find_all('td')]

        if len(values) != len(keys) or values.count('-') > 0:
            return None

        fvalues = []

        def frac_to_float(val):
            return float(val.split('/')[0]) / float(val.split('/')[-1])

        for key, val in zip(keys, values):
            if key == u'팀명':
                fvalues.append(val)
                continue

            try:
                fvalues.append(float(val))
            except:
                if val.find(' ') != -1:
                    fvalues.append(sum([frac_to_float(x) if x.find('/') != -1 else float(x) for x in val.split()]))

        ret.update(dict(zip(keys, fvalues)))

    ret.update(get_basic_info(src))
    return ret

In [17]:
from collections import defaultdict

def merge_dicts(dicts):
    ret = defaultdict(list)

    for d in dicts:
        for k in d.keys():
            ret[k].append(d[k])

    return ret

In [18]:
def get_players_in_team(code):
    ret = {
        'batters': [],
        'pitchers': []
    }

    for men in player_list[code][0]:
        r = get_batter_info(men)
        if r:
            ret['batters'].append(r)
    for men in player_list[code][1]:
        r = get_pitcher_info(men)
        if r:
            ret['pitchers'].append(r)

    ret['batters'] = merge_dicts(ret['batters'])
    ret['pitchers'] = merge_dicts(ret['pitchers'])

    return ret

In [19]:
SS = get_players_in_team('SS')
SSBatter = pd.DataFrame(SS['batters'])
SSPitcher = pd.DataFrame(SS['pitchers'])

NC = get_players_in_team('NC')
NCBatter = pd.DataFrame(NC['batters'])
NCPitcher = pd.DataFrame(NC['pitchers'])

WO = get_players_in_team('WO')
WOBatter = pd.DataFrame(WO['batters'])
WOPitcher = pd.DataFrame(WO['pitchers'])

OB = get_players_in_team('OB')
OBBatter = pd.DataFrame(OB['batters'])
OBPitcher = pd.DataFrame(OB['pitchers'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [20]:
SSBatter.to_csv('data/ss_batters.csv', encoding='euc-kr', index=False)
SSPitcher.to_csv('data/ss_pitchers.csv', encoding='euc-kr', index=False)

NCBatter.to_csv('data/nc_batters.csv', encoding='euc-kr', index=False)
NCPitcher.to_csv('data/nc_pitchers.csv', encoding='euc-kr', index=False)

WOBatter.to_csv('data/nx_batters.csv', encoding='euc-kr', index=False)
WOPitcher.to_csv('data/nx_pitchers.csv', encoding='euc-kr', index=False)

OBBatter.to_csv('data/ds_batters.csv', encoding='euc-kr', index=False)
OBPitcher.to_csv('data/ds_pitchers.csv', encoding='euc-kr', index=False)

# SK 환화 KIA 롯데 LG KT

In [21]:
SK = get_players_in_team('SK')
SKBatter = pd.DataFrame(SK['batters'])
SKPitcher = pd.DataFrame(SK['pitchers'])

HH = get_players_in_team('HH')
HHBatter = pd.DataFrame(HH['batters'])
HHPitcher = pd.DataFrame(HH['pitchers'])

HT = get_players_in_team('HT')
HTBatter = pd.DataFrame(HT['batters'])
HTPitcher = pd.DataFrame(HT['pitchers'])

LT = get_players_in_team('LT')
LTBatter = pd.DataFrame(LT['batters'])
LTPitcher = pd.DataFrame(LT['pitchers'])

LG = get_players_in_team('LG')
LGBatter = pd.DataFrame(LG['batters'])
LGPitcher = pd.DataFrame(LG['pitchers'])

KT = get_players_in_team('KT')
KTBatter = pd.DataFrame(KT['batters'])
KTPitcher = pd.DataFrame(KT['pitchers'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [22]:
SKBatter.to_csv('data/sk_batters.csv', encoding='euc-kr', index=False)
SKPitcher.to_csv('data/sk_pitchers.csv', encoding='euc-kr', index=False)

HHBatter.to_csv('data/hh_batters.csv', encoding='euc-kr', index=False)
HHPitcher.to_csv('data/hh_pitchers.csv', encoding='euc-kr', index=False)

HTBatter.to_csv('data/ht_batters.csv', encoding='euc-kr', index=False)
HTPitcher.to_csv('data/ht_pitchers.csv', encoding='euc-kr', index=False)

LTBatter.to_csv('data/lt_batters.csv', encoding='euc-kr', index=False)
LTPitcher.to_csv('data/lt_pitchers.csv', encoding='euc-kr', index=False)

LGBatter.to_csv('data/lg_batters.csv', encoding='euc-kr', index=False)
LGPitcher.to_csv('data/lg_pitchers.csv', encoding='euc-kr', index=False)

KTBatter.to_csv('data/kt_batters.csv', encoding='euc-kr', index=False)
KTPitcher.to_csv('data/kt_pitchers.csv', encoding='euc-kr', index=False)